Lab 1

In [3]:
import pandas as pd
names = ['Triston Hudgins','Shijo Joseph','Douglas Yip']
teaminfo = pd.DataFrame()
teaminfo['Team'] = names
teaminfo

,Team
0,Triston Hudgins
1,Shijo Joseph
2,Douglas Yip


• Business Understanding (10 points total).
• Describe the purpose of the data set you selected (i.e., why was this data collected in
the first place?). Describe how you would define and measure the outcomes from the
dataset. That is, why is this data important and how do you know if you have mined
useful knowledge from the dataset? How would you measure the effectiveness of a
good prediction algorithm? Be specific

In [2]:
# load the Lehman's baseball dataset
import pandas as pd
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/dk28yip/MSDS7331_lab1/main/Batting.csv') # read in the csv file
df_2015 = df[(df['yearID'] == 2015)]
df_2015.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
99862,aardsda01,2015,1,ATL,NL,33,1,0,0,0,...,0.0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0
99863,abadfe01,2015,1,OAK,AL,62,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
99864,abreujo02,2015,1,CHA,AL,154,613,88,178,34,...,101.0,0.0,0.0,39,140.0,11.0,15.0,0.0,1.0,16.0
99865,achteaj01,2015,1,MIN,AL,11,0,0,0,0,...,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
99866,ackledu01,2015,1,SEA,AL,85,186,22,40,8,...,19.0,2.0,2.0,14,38.0,0.0,1.0,3.0,3.0,3.0


• Data Understanding (80 points total)
• [10 points] Describe the meaning and type of data (scale, values, etc.) for each
attribute in the data file.
• [15 points] Verify data quality: Explain any missing values, duplicate data, and outliers.
Are those mistakes? How do you deal with these problems? Be specific.
• [10 points] Give simple, appropriate statistics (range, mode, mean, median, variance,
counts, etc.) for the most important attributes and describe what they mean or if you
found something interesting. Note: You can also use data from other sources for
comparison. Explain the significance of the statistics run and why they are meaningful.
• [15 points] Visualize the most important attributes appropriately (at least 5 attributes).
Important: Provide an interpretation for each chart. Explain for each attribute why the
chosen visualization is appropriate.
• [15 points] Explore relationships between attributes: Look at the attributes via scatter
plots, correlation, cross-tabulation, group-wise averages, etc. as appropriate. Explain
any interesting relationships.
• [10 points] Identify and explain interesting relationships between features and the class
you are trying to predict (i.e., relationships with variables and the target classification).
• [5 points] Are there other features that could be added to the data or created from
existing features? Which ones?

[10 points] Describe the meaning and type of data (scale, values, etc.) for each
attribute in the data file.

In [4]:
#following code will describe the data
df_2015.describe()

,yearID,stint,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
count,1486.0,1486.000000,1486.000000,1486.000000,1486.000000,1486.000000,1486.000000,1486.000000,1486.000000,1486.000000,1486.000000,1486.000000,1486.000000,1486.000000,1486.000000,1486.000000,1486.000000,1486.000000,1486.000000
mean,2015.0,1.096904,47.465680,111.364738,13.894347,28.335128,5.546433,0.631898,3.303499,13.223419,1.685734,0.716016,9.470390,25.199192,0.639973,1.078062,0.807537,0.829071,2.516151
std,0.0,0.309279,44.777758,174.890616,23.972998,47.498992,9.534636,1.546506,6.866506,23.508546,4.887574,1.756596,17.470518,38.210625,1.971580,2.494026,1.758281,1.730025,4.675541
min,2015.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2015.0,1.000000,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2015.0,1.000000,31.000000,12.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2015.0,1.000000,68.000000,155.000000,17.000000,39.750000,7.000000,0.000000,3.000000,17.000000,1.000000,0.000000,12.000000,36.000000,0.000000,1.000000,1.000000,1.000000,3.000000
max,2015.0,3.000000,162.000000,638.000000,122.000000,205.000000,45.000000,15.000000,47.000000,130.000000,58.000000,20.000000,143.000000,208.000000,29.000000,30.000000,14.000000,11.000000,28.000000


In [18]:
print (df_2020.dtypes)
print (df_2020.info())

playerID     object
yearID        int64
stint         int64
teamID       object
lgID         object
G             int64
AB            int64
R             int64
H             int64
2B            int64
3B            int64
HR            int64
RBI         float64
SB          float64
CS          float64
BB            int64
SO          float64
IBB         float64
HBP         float64
SH          float64
SF          float64
GIDP        float64
dtype: object
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1360 entries, 107429 to 108788
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   playerID  1360 non-null   object 
 1   yearID    1360 non-null   int64  
 2   stint     1360 non-null   int64  
 3   teamID    1360 non-null   object 
 4   lgID      1360 non-null   object 
 5   G         1360 non-null   int64  
 6   AB        1360 non-null   int64  
 7   R         1360 non-null   int64  
 8   H         1360 non-null   int64  
 9 

(1360, 22)

In [19]:
display(df_2020.shape)

(1360, 22)

The dataset has 1360 rows and 22 columns

# Exceptional Work (10 points total)

In [ ]:
You have free reign to provide additional analyses.
One idea: implement dimensionality reduction, then visualize and interpret the results.

In [1]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import StandardScaler
from matplotlib.pyplot import scatter
import plotly
from plotly.graph_objs import Scatter, Marker, Layout, layout,XAxis, YAxis, Bar, Line
%matplotlib inline

In [ ]:
# Use PCA from sklearn.decomposition
pca = PCA(n_components=2)
lda = LDA(n_components=2)
# Fit data and transform it
pca.fit(X)
X_pca = pca.transform(X)
X_lda = lda.fit(X, Y).transform(X)

In [ ]:
print ('pca:', pca.components_)
print ('lda:', lda.scalings_.T)